In [1]:
import yaml
import numpy as np

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import directed_hausdorff

from data.dataset import NUFTDataset, PolygonDataset
from train.nuft_trainer import Trainer
from model.nn import build_model
from torchinfo import summary

/data/anaconda3/envs/gdl/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
with open('cfg/nuft.yaml', 'r') as f:
    cfg = yaml.safe_load(f)  
    
import pandas as pd
glyph_df = pd.read_pickle(cfg['test'])
glyph_df_o = glyph_df[glyph_df.trans == 'o'].reset_index(drop=True)
glyph_df_r = glyph_df[glyph_df.trans == 'r'].reset_index(drop=True)
glyph_df_sk = glyph_df[glyph_df.trans == 'sk'].reset_index(drop=True)
glyph_df_sc = glyph_df[glyph_df.trans == 'sc'].reset_index(drop=True)

glyph_set_o = NUFTDataset(glyph_df_o).to_data_list()
glyph_set_r = NUFTDataset(glyph_df_r).to_data_list()
glyph_set_sk = NUFTDataset(glyph_df_sk).to_data_list()
glyph_set_sc = NUFTDataset(glyph_df_sc).to_data_list()

glyph_set_o_poly = PolygonDataset(glyph_df_o)
glyph_set_r_poly = PolygonDataset(glyph_df_r)
glyph_set_sk_poly = PolygonDataset(glyph_df_sk)
glyph_set_sc_poly = PolygonDataset(glyph_df_sc)

osm_df = pd.read_pickle(cfg['osm'])
osm_df_o = osm_df[osm_df.trans == 'o'].reset_index(drop=True)    
osm_df_r = osm_df[osm_df.trans == 'r'].reset_index(drop=True)  

osm_set_o = NUFTDataset(osm_df_o).to_data_list()
osm_set_r = NUFTDataset(osm_df_r).to_data_list()

osm_set_o_poly = PolygonDataset(osm_df_o)
osm_set_r_poly = PolygonDataset(osm_df_r)

melb_df = pd.read_pickle(cfg['melb'])
melb_set = NUFTDataset(melb_df).to_data_list()
melb_set_poly = PolygonDataset(melb_df)

  0%|          | 0/49713 [00:00<?, ?it/s]/data/anaconda3/envs/gdl/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 37139/37139 [00:45<00:00, 817.42it/s]


In [4]:
backbone = ['nuft']
ckpt = 'epoch100'

datasets = {'glyph_set_o':glyph_set_o,
            'glyph_set_r':glyph_set_r,
            'glyph_set_sk': glyph_set_sk,
            'glyph_set_sc': glyph_set_sc,
            'osm_set_o': osm_set_o, 
            'osm_set_r': osm_set_r,
            'melb_set': melb_set}

poly_datasets = {'glyph_set_o': glyph_set_o_poly,
                 'glyph_set_r': glyph_set_r_poly,
                 'glyph_set_sk': glyph_set_sk_poly,
                 'glyph_set_sc': glyph_set_sc_poly,
                 'osm_set_o': osm_set_o_poly, 
                 'osm_set_r': osm_set_r_poly,
                 'melb_set': melb_set_poly}

In [5]:

cfg['nn'] = 'nuft'
cfg['path'] = f'save/{cfg["nn"]}'

model = build_model(cfg=cfg)
trainer = Trainer(cfg=cfg) 
model = trainer.load_ckpt(model, 'epoch100')
model.eval()
print(summary(model))

metrics = {}
for key, dataset in datasets.items():
    zs = []
    for data in dataset:
        data.to('cuda')
        z = model.encoder(data.x, data.batch)
        zs.append(z.mean(0).detach().cpu().numpy())

    zs = np.stack(zs, axis=0)
    zs = zs[~np.isnan(zs).any(axis=1)]
    
    # NN neighbors
    nbrs = NearestNeighbors(n_neighbors=7, algorithm='auto').fit(zs)
    dist, idx = nbrs.kneighbors(zs)
    # Hausdorff dist    
    query_id = idx[:, 0]
    target_id = idx[:, 1:]
    haus_dist = []
    
    #Polygon Dataset
    poly_dataset = poly_datasets[key]
    
    for id, tids in enumerate(target_id):
        qid = query_id[id]
        tem_dist = []
        for tid in tids: 
            dist = directed_hausdorff(poly_dataset[qid].pos.cpu().numpy(), 
                                      poly_dataset[tid].pos.cpu().numpy())
            tem_dist.append(dist[0])
        haus_dist.append(tem_dist)
    metrics[key] =  np.stack(haus_dist, axis=0).mean(axis=0).tolist()
    
nuft = {'nuft': metrics}
nuft

Layer (type:depth-idx)                   Param #
NUFTAE                                   --
├─MLP: 1-1                               --
│    └─ReLU: 2-1                         --
│    └─ModuleList: 2-2                   --
│    │    └─Linear: 3-1                  18,496
│    │    └─Linear: 3-2                  4,160
│    │    └─Linear: 3-3                  4,160
│    └─ModuleList: 2-3                   --
│    │    └─BatchNorm: 3-4               128
│    │    └─BatchNorm: 3-5               128
├─MLP: 1-2                               --
│    └─ReLU: 2-4                         --
│    └─ModuleList: 2-5                   --
│    │    └─Linear: 3-6                  4,160
│    │    └─Linear: 3-7                  4,160
│    │    └─Linear: 3-8                  18,720
│    └─ModuleList: 2-6                   --
│    │    └─BatchNorm: 3-9               128
│    │    └─BatchNorm: 3-10              128
Total params: 54,368
Trainable params: 54,368
Non-trainable params: 0


{'nuft': {'glyph_set_o': [0.3040210792059363,
   0.3388105981728668,
   0.3541433089953008,
   0.3653348199277107,
   0.37135671788542,
   0.37947436872197354],
  'glyph_set_r': [0.417412421093526,
   0.4421264620275741,
   0.4567647633201862,
   0.4630268230433333,
   0.47006510950496916,
   0.4721402989419446],
  'glyph_set_sk': [0.4655259770041129,
   0.4885836485342498,
   0.495992625322638,
   0.5016636684186228,
   0.5022235198747118,
   0.505781757872477],
  'glyph_set_sc': [0.3368635281557071,
   0.3606142331962186,
   0.37437789519445586,
   0.37993005255077095,
   0.3905185197995516,
   0.39342061051064386],
  'osm_set_o': [0.3020906620380905,
   0.3274105707805531,
   0.34121708489604086,
   0.3528130065962359,
   0.35411628105851184,
   0.363284530014798],
  'osm_set_r': [0.43173694192694867,
   0.46108942178091566,
   0.4813476718854383,
   0.4910040714818395,
   0.5021763593923436,
   0.5083815248052662],
  'melb_set': [0.3827909689520503,
   0.4015262199702834,
   0.4131